In [ ]:
!pip install sqlite3

In [15]:
def search_by_filename(conn, filename):
    cursor = conn.cursor()
    query = '''
    SELECT 
        f.file_id,
        f.name AS filename,
        GROUP_CONCAT(DISTINCT k.keyw_id) AS keyword_ids,
        GROUP_CONCAT(DISTINCT mv.value) AS metadata_values
    FROM 
        files f
    LEFT JOIN 
        keywords k ON f.file_id = k.file_id
    LEFT JOIN 
        metadata m ON f.file_id = m.file_id
    LEFT JOIN 
        metadata_values mv ON m.value_id = mv.id
    WHERE 
        f.name = ?
    GROUP BY
        f.file_id, f.name
    '''
    cursor.execute(query, (filename,))
    return cursor.fetchall()

def process_search_results(filename, conn):
    results = search_by_filename(conn, filename)

    if results:
        print(f"Results for '{filename}':")
        for row in results:
            print(f"File ID: {row[0]}, Filename: {row[1]}")
            print("Keyword IDs:", row[2])
            print("Metadata Values:", row[3])
            
            metadata_values = row[3].split(',')
            
            # Extract Character keywords and remove prefixes
            character_keywords = [value.strip().split('|')[1] for value in metadata_values if value.strip().startswith('Character|')]
            
            # Extract and separate Sounds keywords
            sounds_keywords = [value.strip().split('|')[1:] for value in metadata_values if value.strip().startswith('Sounds|')]
            sounds1_keywords = list(set(kw[0] for kw in sounds_keywords))
            sounds2_keywords = list(set(kw[1] for kw in sounds_keywords if len(kw) > 1))
            
            # Extract base instrument
            base_instrument = next((value.strip() for value in metadata_values if value.strip() in ['Wavetable', 'Analog', 'Operator', 'Sampler']), None)
            
            # Determine if it's an instrument rack
            is_instrument_rack = 'Instrument Rack' in metadata_values
            
            print("\nCharacter Keywords:", character_keywords)
            print("Sounds1 Keywords:", sounds1_keywords)
            print("Sounds2 Keywords:", sounds2_keywords)
            print("Base Instrument:", base_instrument)
            print("Is Instrument Rack:", is_instrument_rack)
            print()
            return sounds1_keywords, sounds2_keywords, character_keywords 
    else:
        print(f"No results found for '{filename}'")



In [19]:
filename = 'Chord Anthem Pad.adg'
_,_,_ = process_search_results(filename, conn)

Results for 'Chord Anthem Pad.adg':
File ID: 5148, Filename: Chord Anthem Pad.adg
Keyword IDs: 8581,8628
Metadata Values: Created by: Philipp Matalla,device:ableton:instr:InstrumentGroupDevice,device:ableton:instr:InstrumentVector,Instrument Rack,Wavetable,Instrument,Character|Chord,Sounds|Pad

Character Keywords: ['Chord']
Sounds1 Keywords: ['Pad']
Sounds2 Keywords: []
Base Instrument: Wavetable
Is Instrument Rack: True



In [20]:
import sqlite3
import json
import os

# Create a connection to the SQLite database
conn = sqlite3.connect('./Live-files-1218.db')
print("Connected to the database successfully.")

# List to record filenames with no results
no_results_files = []

# Function to process each file
def process_file(filepath):
    with open(filepath, 'r') as file:
        labeled_data = json.load(file)

    filename = os.path.basename(filepath).replace('.json', '.adv')
    try:
        s1kw, s2kw, chkw = process_search_results(filename, conn)
    except TypeError:
        filename = os.path.basename(filepath).replace('.json', '.adg')
        try:
            s1kw, s2kw, chkw = process_search_results(filename, conn)
        except TypeError:
            no_results_files.append(filename)
            print(f"No results found for '{filename}'")
            return

    # Remove duplicates before updating labeled data
    labeled_data['sounds'] = list(set(s1kw + s2kw))
    labeled_data['keywords']['character'] = list(set(labeled_data['keywords']['character']).union(chkw))

    # Save the updated labeled data back to the JSON file
    with open(filepath, 'w') as file:
        json.dump(labeled_data, file, indent=4)

    print(f"Updated labeled data saved successfully for {filepath}.")

# Process all JSON files in the ./labeled/ directory
labeled_dir = './labeled/'
for filename in os.listdir(labeled_dir):
    if filename.endswith('.json'):
        process_file(os.path.join(labeled_dir, filename))

# Print filenames with no results
if no_results_files:
    print("No results found for the following files:")
    for file in no_results_files:
        print(file)

Connected to the database successfully.
No results found for 'Mindfulness.adv'
Results for 'Mindfulness.adg':
File ID: 5313, Filename: Mindfulness.adg
Keyword IDs: 8620,8628,8643,8689,8745
Metadata Values: Created by: Francis Preve,device:ableton:instr:InstrumentGroupDevice,device:ableton:instr:InstrumentVector,device:ableton:instr:Operator,Instrument Rack,Operator,Wavetable,Instrument,Character|Bright,Character|Digital,Character|Evolving,Sounds|Ambience & FX|Atmosphere,Sounds|Pad

Character Keywords: ['Bright', 'Digital', 'Evolving']
Sounds1 Keywords: ['Ambience & FX', 'Pad']
Sounds2 Keywords: ['Atmosphere']
Base Instrument: Operator
Is Instrument Rack: True

Updated labeled data saved successfully for ./labeled/Mindfulness.json.
No results found for 'Starshower Pad.adv'
Results for 'Starshower Pad.adg':
File ID: 5205, Filename: Starshower Pad.adg
Keyword IDs: 8628,8742
Metadata Values: Created by: Huston Singletary 

,device:ableton:instr:InstrumentGroupDevice,device:ableton:instr:In

In [21]:
import json
import os

# Function to combine all labeled data into a single JSON file
def combine_labeled_data(labeled_dir, output_file):
    combined_data = []

    # Process all JSON files in the labeled directory
    for filename in os.listdir(labeled_dir):
        if filename.endswith('.json'):
            filepath = os.path.join(labeled_dir, filename)
            with open(filepath, 'r') as file:
                labeled_data = json.load(file)
                combined_data.append(labeled_data)

    # Save combined data to a new JSON file
    with open(output_file, 'w') as file:
        json.dump(combined_data, file, indent=4)

    print(f"Combined labeled data saved successfully to '{output_file}'.")

# Directory containing labeled JSON files
labeled_dir = './labeled/'
# Output file for combined data
output_file = './wavetable_presets.json'

# Combine labeled data
combine_labeled_data(labeled_dir, output_file)

Combined labeled data saved successfully to './wavetable_presets.json'.


In [25]:
import os
import shutil

# Function to copy instrument files to a new folder
def copy_instrument_files(labeled_dir, source_dir, dest_dir):
    # Ensure the destination directory exists
    os.makedirs(dest_dir, exist_ok=True)

    # Function to find a file in the source directory and its subdirectories
    def find_file(filename, source_dir):
        for root, _, files in os.walk(source_dir):
            if filename in files:
                return os.path.join(root, filename)
        return None

    # Process all JSON files in the labeled directory
    for filename in os.listdir(labeled_dir):
        if filename.endswith('.json'):
            base_filename = os.path.basename(filename).replace('.json', '')
            adv_filename = f"{base_filename}.adv"
            adg_filename = f"{base_filename}.adg"

            print(f"Searching for '{adv_filename}' and '{adg_filename}' in '{source_dir}'")

            # Attempt to find and copy .adv file
            adv_source_path = find_file(adv_filename, source_dir)
            if adv_source_path:
                shutil.copy(adv_source_path, dest_dir)
                print(f"Copied '{adv_filename}' to '{dest_dir}'")
            else:
                # Attempt to find and copy .adg file if .adv file does not exist
                adg_source_path = find_file(adg_filename, source_dir)
                if adg_source_path:
                    shutil.copy(adg_source_path, dest_dir)
                    print(f"Copied '{adg_filename}' to '{dest_dir}'")
                else:
                    print(f"No .adv or .adg file found for '{base_filename}'")

# Directory containing labeled JSON files
labeled_dir = './labeled/'
# Source directory containing instrument files
source_dir = './Live 12 Instruments/'
# Destination directory for copied files
dest_dir = './raw/'

# Copy instrument files
copy_instrument_files(labeled_dir, source_dir, dest_dir)

Searching for 'Mindfulness.adv' and 'Mindfulness.adg' in './Live 12 Instruments/'
Copied 'Mindfulness.adg' to './raw/'
Searching for 'Starshower Pad.adv' and 'Starshower Pad.adg' in './Live 12 Instruments/'
Copied 'Starshower Pad.adg' to './raw/'
Searching for 'Basic Bells.adv' and 'Basic Bells.adg' in './Live 12 Instruments/'
Copied 'Basic Bells.adg' to './raw/'
Searching for 'Position Glass.adv' and 'Position Glass.adg' in './Live 12 Instruments/'
Copied 'Position Glass.adv' to './raw/'
Searching for 'Hit Slide Pad.adv' and 'Hit Slide Pad.adg' in './Live 12 Instruments/'
Copied 'Hit Slide Pad.adg' to './raw/'
Searching for 'Venus Doppler.adv' and 'Venus Doppler.adg' in './Live 12 Instruments/'
Copied 'Venus Doppler.adv' to './raw/'
Searching for 'Alphalog Brass.adv' and 'Alphalog Brass.adg' in './Live 12 Instruments/'
Copied 'Alphalog Brass.adv' to './raw/'
Searching for 'Island Bells.adv' and 'Island Bells.adg' in './Live 12 Instruments/'
Copied 'Island Bells.adg' to './raw/'
Search